In [ ]:
#TDA API Greek Options Attribution Tool

### Data Aquisition

- Aquiring the data from TDA's API. 
- I'll have to use my TDA credentials, so I'm hiding them inside my env file and only using them as a specific variable. That way the actual credentials shouldn't be compromised.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime

import env

import tda
import os
import requests
import json

Polygon.io API key loaded up.
Cboe API key loaded up.
TD Ameritrade API key loaded up.
Coinbase API key loaded up.
All credentials loaded successfully


In [3]:
# TDA API keys

tda_key = env.tda_api_key
tda_url = env.my_redirect_url

In [6]:
# Cboe LiveVol API keys:

cboe_key = env.cboe_client_id
cboe_secret = env.cboe_client_secret

In [7]:
requests.get(f'https://api.tdameritrade.com//v1/marketdata/chains?apikey={tda_key}&symbol=AAPL&contractType=CALL&strikeCount=2&includeQuotes=TRUE&strategy=SINGLE&toDate=2021-03-16&expMonth=MAR&optionType=S HTTP/1.1')

<Response [200]>

In [8]:
aapl_api_test = requests.get(f'https://api.tdameritrade.com//v1/marketdata/chains?apikey={tda_key}&symbol=AAPL&contractType=CALL&strikeCount=2&includeQuotes=TRUE&strategy=SINGLE&toDate=2021-03-16&expMonth=MAR&optionType=S HTTP/1.1')

In [9]:
aapl_content = aapl_api_test.content

# aapl_content

In [19]:
request = requests.get(f'https://api.tdameritrade.com//v1/marketdata/chains?apikey={tda_key}&symbol=AAPL&contractType=CALL&strikeCount=2&includeQuotes=TRUE&strategy=SINGLE&toDate=2021-03-16&expMonth=MAR&optionType=S HTTP/1.1').text
aapl = json.loads(request)
# print(aapl['callExpDateMap'])

In [20]:
aapl

{'symbol': 'AAPL',
 'status': 'SUCCESS',
 'underlying': {'symbol': 'AAPL',
  'description': 'Apple Inc. - Common Stock',
  'change': 0.0,
  'percentChange': 0.0,
  'close': 129.87,
  'quoteTime': 1613782799834,
  'tradeTime': 1613782798447,
  'bid': 129.72,
  'ask': 129.74,
  'last': 129.87,
  'mark': 129.87,
  'markChange': 0.0,
  'markPercentChange': 0.0,
  'bidSize': 1100,
  'askSize': 1000,
  'highPrice': 130.71,
  'lowPrice': 128.8,
  'openPrice': 130.24,
  'totalVolume': 87668834,
  'exchangeName': 'NAS',
  'fiftyTwoWeekHigh': 145.09,
  'fiftyTwoWeekLow': 53.1525,
  'delayed': True},
 'strategy': 'SINGLE',
 'interval': 0.0,
 'isDelayed': True,
 'isIndex': False,
 'interestRate': 0.1,
 'underlyingPrice': 129.73000000000002,
 'volatility': 29.0,
 'daysToExpiration': 0.0,
 'numberOfContracts': 4,
 'callExpDateMap': {'2021-03-05:13': {'129.0': [{'putCall': 'CALL',
     'symbol': 'AAPL_030521C129',
     'description': 'AAPL Mar 5 2021 129 Call (Weekly)',
     'exchangeName': 'OPR',
  

In [ ]:
aapl['callExpDateMap']

In [ ]:
aapl['callExpDateMap']['2021-03-05:13']['129.0'][0]

In [ ]:
aapl['callExpDateMap']['2021-03-05:13']['129.0']

### Preparing the data

- The goal is to grab just the option greek and volatility data from the option chain. We don't need any other data at this point.
- I'm going to use a Pandas dataframe to help organize my work.

In [22]:
df = pd.DataFrame(aapl['callExpDateMap']['2021-03-05:13']['129.0'][0], index = [0])

df

,putCall,symbol,description,exchangeName,bid,ask,last,mark,bidSize,askSize,...,multiplier,settlementType,deliverableNote,isIndexOption,percentChange,markChange,markPercentChange,mini,inTheMoney,nonStandard
0,CALL,AAPL_030521C129,AAPL Mar 5 2021 129 Call (Weekly),OPR,3.2,3.25,3.2,3.23,1,1,...,100.0,,,None,-10.62,0.02,0.78,False,True,False


In [23]:
df.columns

Index(['putCall', 'symbol', 'description', 'exchangeName', 'bid', 'ask',
       'last', 'mark', 'bidSize', 'askSize', 'bidAskSize', 'lastSize',
       'highPrice', 'lowPrice', 'openPrice', 'closePrice', 'totalVolume',
       'tradeDate', 'tradeTimeInLong', 'quoteTimeInLong', 'netChange',
       'volatility', 'delta', 'gamma', 'theta', 'vega', 'rho', 'openInterest',
       'timeValue', 'theoreticalOptionValue', 'theoreticalVolatility',
       'optionDeliverablesList', 'strikePrice', 'expirationDate',
       'daysToExpiration', 'expirationType', 'lastTradingDay', 'multiplier',
       'settlementType', 'deliverableNote', 'isIndexOption', 'percentChange',
       'markChange', 'markPercentChange', 'mini', 'inTheMoney', 'nonStandard'],
      dtype='object')

In [24]:
greeks = ['delta', 'gamma', 'theta', 'vega', 'rho']

volatility_info = ['openInterest', 'timeValue', 'theoreticalOptionValue', 'theoreticalVolatility']

basic_info = ['mark', 'totalVolume', 'volatility', 'inTheMoney']

In [25]:
greeks + basic_info + volatility_info

['delta',
 'gamma',
 'theta',
 'vega',
 'rho',
 'mark',
 'totalVolume',
 'volatility',
 'inTheMoney',
 'openInterest',
 'timeValue',
 'theoreticalOptionValue',
 'theoreticalVolatility']

In [26]:
columns_list = basic_info + greeks + volatility_info

In [27]:
df[[c for c in df.columns if c in columns_list]]

,mark,totalVolume,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,theoreticalVolatility,inTheMoney
0,3.23,2620,26.48,0.561,0.057,-0.093,0.103,0.028,4107,2.33,3.2,29.0,True


In [28]:
df = df[[c for c in df.columns if c in columns_list]]
df

,mark,totalVolume,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,theoreticalVolatility,inTheMoney
0,3.23,2620,26.48,0.561,0.057,-0.093,0.103,0.028,4107,2.33,3.2,29.0,True


In [29]:
# 3.53 to 2.50



In [33]:
livevol_url = 'https://api.livevol.com/v1/delayed/allaccess/market/option-and-underlying-quotes?root=AAPL&option_type=C&date=2021-02-17&min_expiry=2021-03-19&max_expiry=2021-03-19&min_strike=130&max_strike=130&symbol=AAPL'

In [35]:
import requests

In [36]:
requests.get('https://api.livevol.com/v1/delayed/allaccess/market/option-and-underlying-quotes?root=AAPL&option_type=C&date=2021-02-17&min_expiry=2021-03-19&max_expiry=2021-03-19&min_strike=130&max_strike=130&symbol=AAPL')

<Response [403]>

In [ ]:
requests.get('https://sandbox.livevol.com/api/v1/delayed/market/symbols/AAPL/options?root=AAPL&expiry=2021-02-19&strike=130&option_type=C&date=2021-02-10')

In [32]:
import json
import base64
import requests
# Required package requests: `pip install requests`

client_id = cboe_key
client_secret = cboe_secret

identity_url = "https://id.livevol.com/connect/token"

encoded = base64.b64encode((client_id + ':' + client_secret).encode())
headers = {"Authorization": "Basic " + encoded.decode('ascii')}
payload = {"grant_type": "client_credentials"}

# Requesting access token
token_data = requests.post(identity_url, data=payload, headers=headers)

if token_data.status_code == 200:
    token = token_data.json()['access_token']
    if len(token) > 0:
        print("Authenticated successfully")
        # Requesting data from API
        result = requests.get("https://api.livevol.com/v1/delayed/market/symbols/AAPL", headers={"Authorization": "Bearer " + token}, verify=False)
        print(result.json())
else:
    print("Authentication failed")

Authenticated successfully


/usr/local/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.livevol.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# https://www.livevol.com/apis/technical-reference-legacy?m=market-at-a-glance/options

In [ ]:
# https://www.livevol.com/apis/technical-reference/?m=market-at-a-glance/option-and-underlying-quotes